# Мультиклассовая классификация

В данном ноутбуке рассмотрим подход к задаче с точки зрения мультиклассовой классификации. То есть, мы не будем учитывать порядок классов, а попробуем просто спрогнозировать их, как равнозначные.

В ноутбуке с анализом данных, исходя из корреляций признаков объектов и целевого значения, удалось установить, что многие признаки имеют высокую корреляцию с таргетом. Таким образом, стоит попробовать модель LogisticRegression.


In [1]:
import pandas as pd
import numpy as np

In [5]:
df_train = pd.read_csv("/content/drive/MyDrive/intern_task_train.csv")
df_test = pd.read_csv("/content/drive/MyDrive/intern_task_test.csv")

Предварительно создадим функцию для оценки требуемой метрики ранжирования NDCG_5.

Из анализа датасета, удалось установить, что количество документов, соответсвующих поисковой сессии, не имет фиксированную длину. А также может быть меньше 5.

In [16]:
from sklearn.metrics import ndcg_score
from sklearn.metrics import accuracy_score


def NDCG_atK_score(test_Data, pred_Data, k, logits=False):
  query_ids = test_Data['query_id'].unique()
  test_Data_ranks = np.asarray([[test_Data[test_Data['query_id'] == id]['rank'].iloc[i] for i in range(min(len(test_Data[test_Data['query_id'] == id]), k))] + [0 for _ in range(max(0, k - len(test_Data[test_Data['query_id'] == id])))] for id in query_ids])
  pred_Data_ranks = np.asarray([[pred_Data[pred_Data['query_id'] == id]['rank'].iloc[i] for i in range(min(len(pred_Data[pred_Data['query_id'] == id]), k))] + [0 for _ in range(max(0, k - len(pred_Data[pred_Data['query_id'] == id])))] for id in query_ids])
  if logits:
    print(test_Data_ranks)
    print(pred_Data_ranks)
  return ndcg_score(test_Data_ranks, pred_Data_ranks)

Подготовим данные: оставим из сильно коррелирующих признаков объектов только один признак (такие признаки были установлены при анализе датасета). Применим стандартизацию и используем OHE для кодирования категориальных признаков.

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

to_drop = ['feature_14', 'feature_17', 'feature_19', 'feature_20',
       'feature_29', 'feature_34', 'feature_35', 'feature_44',
       'feature_74', 'feature_79', 'feature_84', 'feature_89',
       'feature_94', 'feature_114', 'feature_119', 'feature_120',
       'feature_121', 'feature_122', 'feature_123', 'feature_124',
       'feature_142', 'feature_143']

cat_features = ['feature_95', 'feature_96', 'feature_97', 'feature_98', 'feature_99']

df_train = pd.get_dummies(df_train, columns=cat_features)
df_test = pd.get_dummies(df_test, columns=cat_features)

X_train = df_train.drop(columns=to_drop)
X_train = X_train.drop(columns=['rank', 'query_id'])
X_test = df_test.drop(columns=to_drop)
X_test = X_test.drop(columns=['rank', 'query_id'])
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [11]:
from sklearn.linear_model import LogisticRegression

In [13]:
logreg = LogisticRegression(max_iter=1500, class_weight='balanced')
logreg.fit(X_train, df_train['rank'])

LogisticRegression(class_weight='balanced', max_iter=1500)

In [14]:
outputs = logreg.predict(X_test)
df_tmp = df_test.copy()
df_tmp['rank'] = outputs
print("NDCG@5 score : ", NDCG_atK_score(df_test, df_tmp, k=5))
print("NDCG@10 score : ", NDCG_atK_score(df_test, df_tmp, k=10))


0.9140344866188053
0.8837049835008765


Так как NDCG@k принимает значения из диапазона $[0, 1]$, при этом лучше, чем ближе к 1, то можно сказать, что модель справилась хорошо. Однако, если не ограничиваться k=5, а посмотреть, например, на k=10, то заметим, что показатель падает.

In [17]:
NDCG_atK_score(df_test, df_tmp, k=10, logits=True)

[[1 1 1 ... 1 1 1]
 [4 3 2 ... 2 2 2]
 [4 3 2 ... 1 1 1]
 ...
 [4 4 4 ... 4 4 4]
 [3 3 3 ... 2 2 2]
 [3 2 2 ... 1 1 1]]
[[0 0 0 ... 0 0 2]
 [4 0 2 ... 3 2 2]
 [4 2 0 ... 1 1 1]
 ...
 [4 4 4 ... 4 4 1]
 [3 3 4 ... 4 4 2]
 [2 2 1 ... 2 2 2]]


0.8837049835008765

### Вывод

Данный подход позволил получить вполне хорошие результаты. Конечно модель ошибается, предсказывая объекту более низкий (или, наоборот, более высокий) ранг.
Проблема данного подхода в том, что мы рассматриваем ранги, как равноправные классы, хотя эти метки имеют четку упорядоченную структуру (условно, ранг 3 между рангами 2 и 4).